In [7]:
# Use

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm, tqdm_pandas, tqdm_notebook


In [9]:
tqdm.pandas()

In [10]:
!ls | grep bios


3-cyy5-get-bios.ipynb
6-cy-get-harvard-bios.ipynb
7-cy-parse-yale-bios-Copy1.ipynb
7-cy-parse-yale-bios.ipynb
bios.json
bios.lzw
bios.p
harvard_bios.csv
harvard_bios.csv.gz
yale_bios.csv.gz


In [11]:

df = pd.read_csv('harvard_bios.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10806 entries, 0 to 10805
Data columns (total 2 columns):
Unnamed: 0    10806 non-null object
0             10774 non-null object
dtypes: object(2)
memory usage: 168.9+ KB


In [12]:
df.head()

,Unnamed: 0,0
0,/sports/bsb/2008-09/bios/albright_tyler,"<div class=""bio-wrap clearfix"" data-module=""st..."
1,/sports/bsb/2008-09/bios/arledge_cole,"<div class=""bio-wrap clearfix"" data-module=""st..."
2,/sports/bsb/2008-09/bios/hofeld_zach,"<div class=""bio-wrap clearfix"" data-module=""st..."
3,/sports/bsb/2016-17/bios/allen_jake_gyf5,"<div class=""bio-wrap bio-wrap-has-highlights c..."
4,/sports/mbkb/1984-85/Bios/Eric_Wanta,"<div class=""bio-wrap clearfix"" data-module=""st..."


We'll need to break up the strings into blocks and only use the markup that we actually need.

In [13]:
# the output needs to be handled 1 at a time, and then dumped out carefully.
# df['soup'] = df['0'].progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

In [14]:
tSoup = df['0'].head().progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

100%|██████████| 5/5 [00:01<00:00,  3.88it/s]


In [15]:
ts = tSoup.iloc[0]

In [16]:
def tableToDict(soup):
    "Given a headerless table, return a dict with all the fields"
    table = {}
    for row in soup.find_all('tr'):
        cells = row.find_all('td')
        table[str(cells[0].text.strip())] = cells[1].text.strip()
        
    return table
    
def soupToDict(soup):
    "Given a player's soup blob, parse their header"
    playerData = tableToDict(soup.select('div.player-info')[0].find('table'))
    playerData['name'] = soup.select('div.player-name span.name')[0].text.strip()
    return playerData
def getSynopsis(soup):
    "Given a player's soup, split their bio information into \
    chunks for each strong header"
    
    # Get all top level children
#     bioPs = soup.select('div.synopsis')[0].find_all('p', recursive=False)
    
#     blocks= []
#     # if their blocks are malformed, skip that block.
#     for i, val in enumerate(bioPs):
#         try:
#             blocks[val.select('strong')[0].text] = val.get_text()
#         except:
#             printf("Text not detected")
        
    
    # Given the variation in blob types, just return
    # single blob for now.
    # Ryan Brenner: b instead of strong tags
    # Jackson Stallings (Junior Year) - UL blobs mixed with P tags
    # Jackson Stallings (Freshman) - Clean example
    
    # Sometimes, no bio
#     http://yalebulldogs.com/sports/c-sail/2016-17/bios/buehler_patrick_nu3o?view=news
    bio = soup.find('div',class_='synopsis')
    return bio.get_text() if bio else None

In [17]:
ts.select('div.synopsis')[0].find_all('b')

[]

In [18]:
getSynopsis(ts)

u"\nJunior (2010):\n\n Named team's captain prior to junior season ... Named Ivy League's\ntop defensive catcher and the top arm by College Baseball Insider\nin the preseason ... Named All-Ivy League honorable mention after\nplaying virtually the entire season with a fractured thumb on his\nglove hand ... One of just six players to hit .400 in league play,\nhitting the mark exactly while posting a .493 on-base-percentage to\nrank fourth in the league ... Earned seven walks and was hit by six\npitches while striking out just three times in 19 games ... Batted\n.426 against righties in league play and was at his best in clutch\nsituations with a .524 average with runners in scoring position\n(11-for-21) ... Drove in 15 runs during league play and led the\nteam with 30 RBI for the season while batting .321 overall with\nnine doubles ... Had three hits at at UNC Greensboro (3/17) ... Had\nthree RBI at UNC Greensboro (3/18) ... Hit a solo homer at Sacred\nHeart (3/27) ... Hit a solo homer a

In [19]:
soupToDict(ts)

{'Bats/Throws:': u'R/R',
 'Height:': u'6-2',
 'High School:': u'Woodrow Wilson',
 'Hometown:': u'Long Beach, Calif.',
 'House Affiliation:': u'Winthrop',
 'Position:': u'C',
 'Weight:': u'190',
 'Year:': u'Jr.',
 'name': u'Tyler Albright'}

In [20]:
# df['types'] = df['0'].map(type)
# df['types'].describe()

In [21]:
# df['types']

In [22]:
df['headers'] = df['0'].progress_apply(lambda x: \
                                       soupToDict(BeautifulSoup(x, 'lxml')) if (type(x) == str) else None)

100%|██████████| 10806/10806 [28:22<00:00,  1.21s/it]


In [23]:
df['synopsis'] = df['0'].progress_apply(lambda x: getSynopsis(BeautifulSoup(x, 'lxml')) if (type(x) == str) else None)

100%|██████████| 10806/10806 [26:59<00:00,  1.37s/it]


In [27]:
df.index = df['Unnamed: 0']

In [28]:
df.head()

,Unnamed: 0,0,headers,synopsis
Unnamed: 0,,,,
/sports/bsb/2008-09/bios/albright_tyler,/sports/bsb/2008-09/bios/albright_tyler,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Position:': u'C', u'Year:': u'Jr.', u'Heigh...",\nJunior (2010):\n\n Named team's captain prio...
/sports/bsb/2008-09/bios/arledge_cole,/sports/bsb/2008-09/bios/arledge_cole,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Position:': u'C', u'Year:': u'Jr.', u'Heigh...",\nJunior (2010):\n\n Started at catcher agains...
/sports/bsb/2008-09/bios/hofeld_zach,/sports/bsb/2008-09/bios/hofeld_zach,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Position:': u'RHP', u'Year:': u'Jr.', u'Hei...",\nSophomore (2009):\n\nStruck out three batter...
/sports/bsb/2016-17/bios/allen_jake_gyf5,/sports/bsb/2016-17/bios/allen_jake_gyf5,"<div class=""bio-wrap bio-wrap-has-highlights c...","{u'Position:': u'C', u'Year:': u'So.', u'Heigh...",\nCareer Highs\nHits- 1 (twice)\n\nRuns- 1 (tw...
/sports/mbkb/1984-85/Bios/Eric_Wanta,/sports/mbkb/1984-85/Bios/Eric_Wanta,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Position:': u'Forward', u'Year:': u'Freshma...",None


In [29]:
df[['synopsis', 'headers']].to_csv('harvard_bio_parsed.csv', encoding='utf8')

In [31]:
!gzip -c harvard_bio_parsed.csv > harvard_bio_parsed.csv.gz

In [ ]:
# if we redo this, batching it in databricks / spark job will save a lot of time.

In [32]:
!ls -lS | grep parsed

-rw-r--r-- 1 cyy5 cyy5   15549144 Nov 26 01:17 harvard_bio_parsed.csv
-rw-rw-r-- 1 cyy5 cyy5   12321518 Nov 26 01:16 yale_bio_parsed.csv
-rw-r--r-- 1 cyy5 cyy5    3702197 Nov 26 01:21 harvard_bio_parsed.csv.gz
-rw-r--r-- 1 cyy5 cyy5       3755 Nov 25 23:36 yale_bio_parsed.csv.gz
